# 🌍 NEO-CORE Storytelling - Sistema de Coleta e Gerenciamento de Dados

## 📖 Pipeline Completo de Ingestão de Dados da NASA

Este notebook documenta **todas as etapas** do módulo **neo-core**, responsável por coletar, normalizar, persistir e disponibilizar dados de asteroides da NASA para o sistema de Machine Learning.

---

## 📋 Índice das Etapas

1. **Arquitetura e Responsabilidades**
2. **Integração com NASA NeoWs API**
3. **Normalização e Transformação dos Dados**
4. **Persistência no PostgreSQL**
5. **Exportação para CSV (Machine Learning)**
6. **Armazenamento no MinIO (S3)**
7. **API REST para Frontend**
8. **Fluxo Completo End-to-End**

---

## 1️⃣ Arquitetura e Responsabilidades

### 🎯 Missão do neo-core

O **neo-core** é o **módulo de backend principal** do sistema NEO, responsável por:

✅ **Coletar** dados da NASA NeoWs API  
✅ **Normalizar** JSON complexo em estrutura relacional  
✅ **Persistir** no PostgreSQL com Flyway migrations  
✅ **Exportar** dados para CSV (treino de ML)  
✅ **Armazenar** no MinIO (S3-compatible storage)  
✅ **Disponibilizar** via RESTful API para frontend  

### 🏗️ Arquitetura em Camadas

```
┌─────────────────────────────────────────┐
│         NASA NeoWs API                  │
│    https://api.nasa.gov/neo/rest/v1     │
└──────────────┬──────────────────────────┘
               │ HTTP GET /feed
               ▼
┌─────────────────────────────────────────┐
│      NeoWsClient (REST Client)          │  ← MicroProfile REST Client
└──────────────┬──────────────────────────┘
               │ FeedResponse (DTO)
               ▼
┌─────────────────────────────────────────┐
│         NeoService (Business)           │  ← Orquestra importação
│  - importarFeed()                       │
│  - normalizarPersistir()                │
└──────────┬──────────────┬───────────────┘
           │              │
           ▼              ▼
┌──────────────────┐  ┌──────────────────┐
│  NeoRepository   │  │  MinioService    │
│  (PostgreSQL)    │  │  (CSV Storage)   │
└──────────────────┘  └──────────────────┘
           │                     │
           ▼                     ▼
    [PostgreSQL]           [MinIO/S3]
```

### 🛠️ Stack Tecnológica

```
☕ Java 17 + Quarkus        → Framework reativo
🗄️ PostgreSQL + Flyway      → Banco relacional + migrations
🐻 Hibernate ORM (Panache)  → ORM simplificado
📡 MicroProfile REST Client → Integração NASA API
📦 MinIO (AWS S3 SDK)       → Object storage
🎯 JAX-RS (RESTEasy)        → API REST
🔄 Jackson                  → JSON/CSV serialization
```

## 2️⃣ Integração com NASA NeoWs API

### 📡 REST Client (MicroProfile)

Interface declarativa para consumir a API da NASA:

```java
@RegisterRestClient(configKey = "nasa-neows")
public interface NeoWsClient {

    @GET
    @Path("/feed")
    public FeedResponse buscarFeed(
        @QueryParam("start_date") String startDate,
        @QueryParam("end_date") String endDate,
        @QueryParam("api_key") String apiKey
    );
}
```

### ⚙️ Configuração (application.properties)

```properties
# API Key da NASA
nasa.api.key=${NASA_API_KEY:DEMO_KEY}
nasa.base.url=https://api.nasa.gov/neo/rest/v1

# REST Client
quarkus.rest-client.nasa-neows.url=${nasa.base.url}
quarkus.rest-client.nasa-neows.scope=Singleton
```

### 📥 Exemplo de Requisição

```http
GET https://api.nasa.gov/neo/rest/v1/feed
    ?start_date=2025-10-01
    &end_date=2025-10-07
    &api_key=DEMO_KEY
```

### 📊 Estrutura da Resposta (Simplificada)

```json
{
  "element_count": 42,
  "near_earth_objects": {
    "2025-10-01": [
      {
        "id": "54016567",
        "name": "(2020 QG5)",
        "absolute_magnitude_h": 19.5,
        "estimated_diameter": {
          "meters": {
            "estimated_diameter_min": 150.2,
            "estimated_diameter_max": 336.4
          }
        },
        "is_potentially_hazardous_asteroid": false,
        "close_approach_data": [
          {
            "close_approach_date_full": "2025-Oct-01 12:34",
            "relative_velocity": {
              "kilometers_per_second": "13.9"
            },
            "orbiting_body": "Earth"
          }
        ]
      }
    ]
  }
}
```

## 3️⃣ Normalização e Transformação dos Dados

### 🔄 Pipeline de Importação

O método `importarFeed()` orquestra todo o processo:

```java
@ApplicationScoped
public class NeoService {

    @Inject @RestClient
    NeoWsClient neoClient;
    
    @ConfigProperty(name = "nasa.api.key")
    String apiKey;
    
    @Inject
    NeoRepository neoRepo;
    
    @Inject
    ArmazenamentoMinioService minioService;

    @Transactional
    public int importarFeed(LocalDate inicio, LocalDate fim) {
        // 1️⃣ Formatar datas para API NASA
        String start = inicio.format(DateTimeFormatter.ISO_DATE);
        String end = fim.format(DateTimeFormatter.ISO_DATE);
        
        // 2️⃣ Buscar dados da NASA
        FeedResponse feed = neoClient.buscarFeed(start, end, apiKey);

        try {
            // 3️⃣ Serializar JSON para auditoria
            String json = mapper.writeValueAsString(feed);
            
            // 4️⃣ Salvar CSV no MinIO (para ML)
            String s3key = minioService.salvarCsvBruto(json, inicio);

            // 5️⃣ Normalizar e persistir no PostgreSQL
            int inseridos = normalizarPersistir(feed, s3key);

            Log.infof("✅ Importação finalizada. %d NEOs importados", inseridos);
            return inseridos;

        } catch (Exception e) {
            throw new RuntimeException("Falha ao importar/normalizar feed", e);
        }
    }
}
```

### 🧩 Normalização de Estruturas Aninhadas

O JSON da NASA tem **estrutura complexa** (nested objects, arrays). Precisamos **achatar** para o modelo relacional:

```java
@Transactional
protected int normalizarPersistir(FeedResponse feed, String s3key) {
    int count = 0;
    DateTimeFormatter nasaFmt = DateTimeFormatter
        .ofPattern("yyyy-MMM-dd HH:mm", Locale.US);

    // Iterar sobre dias e NEOs
    for (Map.Entry<String, List<FeedResponse.Neo>> dia 
            : feed.nearEarthObjects.entrySet()) {
        
        for (FeedResponse.Neo n : dia.getValue()) {
            
            // 🔍 Upsert: buscar por neoId único
            NeoObject ent = safeFindByNeoId(n.id);
            if (ent == null) {
                ent = new NeoObject();
                ent.neoId = n.id;
            }

            // 📝 Mapear campos básicos
            ent.nome = n.name;
            ent.magnitudeAbsoluta = n.absoluteMagnitudeH;
            ent.ehPotencialmentePerigoso = n.hazardous;

            // 📏 Extrair diâmetro estimado
            if (n.estimatedDiameter != null 
                    && n.estimatedDiameter.meters != null) {
                ent.diametroMinM = n.estimatedDiameter.meters.min;
                ent.diametroMaxM = n.estimatedDiameter.meters.max;
            }

            // 🚀 Extrair dados de aproximação (primeiro registro)
            if (n.closeApproachData != null 
                    && !n.closeApproachData.isEmpty()) {
                var ca = n.closeApproachData.get(0);
                
                ent.planetaAlvo = ca.orbitingBody;

                // Velocidade relativa
                try {
                    ent.velocidadeKmS = (ca.relativeVelocity != null)
                        ? Double.parseDouble(ca.relativeVelocity.kmPerSec)
                        : null;
                } catch (NumberFormatException ex) {
                    Log.warnf("Velocidade inválida para NEO %s", n.id);
                }

                // Data de aproximação
                String ad = ca.approachDateFull;
                if (ad != null && !ad.isBlank()) {
                    try {
                        var ldt = LocalDateTime.parse(ad, nasaFmt);
                        ent.dataPrimeiraAproximacao = 
                            OffsetDateTime.of(ldt, ZoneOffset.UTC);
                    } catch (RuntimeException ex) {
                        Log.warnf("Data inválida para NEO %s: '%s'", n.id, ad);
                    }
                }
            }

            // 🔗 Rastreabilidade (origem do dado)
            ent.origemJsonS3Key = s3key;

            // 💾 Persistir (merge = insert ou update)
            Panache.getEntityManager().merge(ent);
            count++;
        }
    }
    return count;
}
```

### ✅ Tratamento de Erros

- **NumberFormatException**: Velocidade inválida → Log warning + null
- **DateTimeParseException**: Data inválida → Log warning + null
- **Duplicatas**: `merge()` garante upsert automático

## 4️⃣ Persistência no PostgreSQL

### 🗄️ Modelo de Dados (Entity)

```java
@Entity
@Table(name = "neo_object")
public class NeoObject extends PanacheEntityBase {

    @Id 
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    public Long id;

    @Column(name="neo_id", unique = true, nullable = false, length = 40)
    public String neoId;  // ID da NASA (único)

    @Column(name="nome", nullable = false, length = 200)
    public String nome;

    @Column(name="magnitude_absoluta")
    public Double magnitudeAbsoluta;

    @Column(name="diametro_min_m")
    public Double diametroMinM;

    @Column(name="diametro_max_m")
    public Double diametroMaxM;

    @Column(name="eh_potencialmente_perigoso", nullable = false)
    public boolean ehPotencialmentePerigoso;

    @Column(name="data_primeira_aproximacao")
    public OffsetDateTime dataPrimeiraAproximacao;

    @Column(name="velocidade_km_s")
    public Double velocidadeKmS;

    @Column(name="planeta_alvo", length = 64)
    public String planetaAlvo;

    @Column(name="origem_json_s3_key", length=512)
    public String origemJsonS3Key;  // Auditoria!

    @Column(name="criado_em", nullable = false)
    public OffsetDateTime criadoEm = OffsetDateTime.now();
}
```

### 📦 Panache Repository

```java
@ApplicationScoped
public class NeoRepository implements PanacheRepository<NeoObject> {
    
    // Métodos já disponíveis:
    // - findAll()
    // - findById(Long id)
    // - persist(NeoObject entity)
    // - delete(NeoObject entity)
    
    public NeoObject findByNeoId(String neoId) {
        return find("neoId", neoId).firstResult();
    }
}
```

### 🛠️ Flyway Migration (V1__init.sql)

```sql
CREATE TABLE neo_object (
    id BIGSERIAL PRIMARY KEY,
    neo_id VARCHAR(40) UNIQUE NOT NULL,
    nome VARCHAR(200) NOT NULL,
    magnitude_absoluta DOUBLE PRECISION,
    diametro_min_m DOUBLE PRECISION,
    diametro_max_m DOUBLE PRECISION,
    eh_potencialmente_perigoso BOOLEAN NOT NULL DEFAULT FALSE,
    data_primeira_aproximacao TIMESTAMP WITH TIME ZONE,
    velocidade_km_s DOUBLE PRECISION,
    planeta_alvo VARCHAR(64),
    origem_json_s3_key VARCHAR(512),
    criado_em TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW()
);

-- Índices para performance
CREATE INDEX idx_neo_perigoso ON neo_object(eh_potencialmente_perigoso);
CREATE INDEX idx_neo_magnitude ON neo_object(magnitude_absoluta);
CREATE INDEX idx_neo_criado_em ON neo_object(criado_em);
```

### ⚙️ Configuração do Banco

```properties
# PostgreSQL
quarkus.datasource.db-kind=postgresql
quarkus.datasource.username=neo
quarkus.datasource.password=neo
quarkus.datasource.jdbc.url=jdbc:postgresql://localhost:5432/neo

# Hibernate
quarkus.hibernate-orm.database.generation=none

# Flyway
quarkus.flyway.migrate-at-start=true
```

## 5️⃣ Exportação para CSV (Machine Learning)

### 📊 Formato CSV para Treinamento

O modelo de ML precisa dos dados em formato **flat CSV** com as features específicas:

```csv
magnitudeAbsoluta,diametroMinM,diametroMaxM,velocidadeKmS,ehPotencialmentePerigoso
19.5,150.2,336.4,13.9,false
18.2,280.5,627.1,18.3,true
22.1,45.7,102.1,8.2,false
```

### 🔄 Conversão JSON → CSV

```java
@ApplicationScoped
public class ArmazenamentoMinioService {
    
    @Inject
    S3Client s3Client;
    
    @ConfigProperty(name = "neo.minio.bucket")
    String bucket;

    public String salvarCsvBruto(String conteudoJson, LocalDate data) 
            throws Exception {
        
        criarBucket();  // Garante que bucket existe

        // 1️⃣ Deserializar JSON para objeto
        ObjectMapper jsonMapper = new ObjectMapper();
        FeedResponse feed = jsonMapper.readValue(
            conteudoJson, FeedResponse.class
        );

        // 2️⃣ Achatar dados em lista simples
        List<CsvNeoData> csvData = new ArrayList<>();

        if (feed.nearEarthObjects != null) {
            for (Map.Entry<String, List<FeedResponse.Neo>> entry 
                    : feed.nearEarthObjects.entrySet()) {
                
                for (FeedResponse.Neo neo : entry.getValue()) {
                    csvData.add(new CsvNeoData(
                        neo.id,
                        neo.name,
                        neo.absoluteMagnitudeH,
                        (neo.estimatedDiameter != null 
                            && neo.estimatedDiameter.meters != null)
                            ? neo.estimatedDiameter.meters.min
                            : null,
                        (neo.estimatedDiameter != null 
                            && neo.estimatedDiameter.meters != null)
                            ? neo.estimatedDiameter.meters.max
                            : null,
                        neo.hazardous,
                        (neo.closeApproachData != null 
                            && !neo.closeApproachData.isEmpty())
                            ? neo.closeApproachData.get(0).approachDateFull
                            : null,
                        (neo.closeApproachData != null 
                            && !neo.closeApproachData.isEmpty()
                            && neo.closeApproachData.get(0).relativeVelocity != null)
                            ? Double.parseDouble(
                                neo.closeApproachData.get(0)
                                    .relativeVelocity.kmPerSec)
                            : null,
                        (neo.closeApproachData != null 
                            && !neo.closeApproachData.isEmpty())
                            ? neo.closeApproachData.get(0).orbitingBody
                            : null,
                        entry.getKey()  // Data
                    ));
                }
            }
        }

        // 3️⃣ Serializar para CSV
        CsvMapper csvMapper = new CsvMapper();
        CsvSchema schema = csvMapper.schemaFor(CsvNeoData.class)
            .withHeader();
        String csvContent = csvMapper.writer(schema)
            .writeValueAsString(csvData);

        // 4️⃣ Salvar no MinIO
        String key = "neos-" + data.toString() + ".csv";
        s3Client.putObject(
            PutObjectRequest.builder()
                .bucket(bucket)
                .key(key)
                .contentType("text/csv")
                .build(),
            RequestBody.fromString(csvContent, StandardCharsets.UTF_8)
        );

        Log.info("✅ CSV salvo no MinIO: " + key);
        return key;
    }
}
```

### 📝 Classe DTO para CSV

```java
public static class CsvNeoData {
    public String neoId;
    public String nome;
    public Double magnitudeAbsoluta;
    public Double diametroMinM;
    public Double diametroMaxM;
    public boolean ehPotencialmentePerigoso;
    public String dataAproximacao;
    public Double velocidadeKmS;
    public String planetaAlvo;
    public String dataFeed;
    
    // Construtor com todos os campos...
}
```

## 6️⃣ Armazenamento no MinIO (S3)

### 📦 Configuração do MinIO

```properties
# AWS S3 (MinIO)
quarkus.s3.endpoint-override=http://localhost:9000
quarkus.s3.aws.region=us-east-1
quarkus.s3.path-style-access=true
quarkus.s3.aws.credentials.type=static
quarkus.s3.aws.credentials.static-provider.access-key-id=minioadmin
quarkus.s3.aws.credentials.static-provider.secret-access-key=minioadmin

# Bucket customizado
neo.minio.bucket=neo
```

### 🔧 Criação Automática de Bucket

```java
public void criarBucket() {
    try {
        // Verificar se bucket existe
        s3Client.headBucket(
            HeadBucketRequest.builder()
                .bucket(bucket)
                .build()
        );
        Log.infof("Bucket '%s' já existe.", bucket);

    } catch (S3Exception e) {
        if (e.statusCode() == 404) {
            // Criar bucket se não existir
            try {
                s3Client.createBucket(
                    CreateBucketRequest.builder()
                        .bucket(bucket)
                        .build()
                );
                Log.infof("✅ Bucket '%s' criado!", bucket);
            } catch (S3Exception ce) {
                Log.errorf(ce, "❌ Falha ao criar bucket '%s'", bucket);
                throw new RuntimeException("Erro ao criar bucket", ce);
            }
        } else {
            Log.errorf(e, "❌ Erro ao verificar bucket '%s'", bucket);
            throw new RuntimeException("Falha ao verificar bucket", e);
        }
    }
}
```

### 📁 Estrutura de Arquivos no MinIO

```
bucket: neo/
├── neos-2025-10-01.csv
├── neos-2025-10-02.csv
├── neos-2025-10-03.csv
├── ...
└── models/                      ← Usado pelo modelo-ia
    ├── weka-model-20251007-143022.model
    └── weka-model-20251007-143022.header
```

### ✅ Vantagens do MinIO

✅ **S3-compatible**: Fácil migrar para AWS S3 em produção  
✅ **Object storage**: Escalável e distribuído  
✅ **Versionamento**: Dados históricos preservados  
✅ **Desacoplamento**: modelo-ia acessa dados independentemente  
✅ **Auditoria**: Rastreabilidade completa via `origemJsonS3Key`

## 7️⃣ API REST para Frontend

### 🎯 Endpoints Disponíveis

```java
@Path("/api/neos")
@Produces(MediaType.APPLICATION_JSON)
@Consumes(MediaType.APPLICATION_JSON)
public class NeoController {

    @Inject
    NeoService neoService;

    @Inject
    NeoObjectMapper neoMapper;

    // 📋 Listar NEOs com paginação e filtro
    @GET
    public Response listar(
        @QueryParam("pagina") @DefaultValue("0") int pagina,
        @QueryParam("tamanho") @DefaultValue("20") int tamanho,
        @QueryParam("perigoso") Boolean perigoso,
        @Context UriInfo uriInfo
    ) {
        List<NeoObject> lista = (perigoso != null)
            ? neoService.listarPerigosos(pagina, tamanho, perigoso)
            : neoService.listar(pagina, tamanho);

        List<NeoObjectResponse> resp = lista.stream()
            .map(neo -> {
                URI selfUri = uriInfo.getBaseUriBuilder()
                    .path("api/neos")
                    .path(String.valueOf(neo.id))
                    .build();
                return neoMapper.toResponse(neo, selfUri);
            })
            .collect(Collectors.toList());

        return Response.ok().entity(resp).build();
    }

    // 🔍 Obter NEO por ID
    @GET
    @Path("/{id}")
    public Response obter(
        @PathParam("id") Long id, 
        @Context UriInfo uriInfo
    ) {
        NeoObject ent = neoService.obterPorId(id);
        if (ent == null)
            throw new NotFoundException("NEO não encontrado");

        URI selfUri = uriInfo.getBaseUriBuilder()
            .path("api/neos")
            .path(String.valueOf(ent.id))
            .build();

        NeoObjectResponse resp = neoMapper.toResponse(ent, selfUri);
        return Response.ok(resp).build();
    }

    // ➕ Criar NEO manualmente
    @POST
    @Transactional
    public Response criar(
        NeoObject neo, 
        @Context UriInfo uriInfo
    ) {
        neoService.criar(neo);

        URI self = uriInfo.getAbsolutePathBuilder()
            .path(String.valueOf(neo.id))
            .build();

        NeoObjectResponse resp = neoMapper.toResponse(neo, self);
        return Response.created(self).entity(resp).build();
    }

    // ✏️ Atualizar NEO
    @PUT
    @Path("/{id}")
    @Transactional
    public Response atualizar(
        @PathParam("id") Long id, 
        NeoObject neo,
        @Context UriInfo uriInfo
    ) {
        NeoObject ent = neoService.obterPorId(id, neo);
        URI uri = uriInfo.getAbsolutePathBuilder()
            .path(String.valueOf(ent.id))
            .build();
        return Response.accepted(uri).entity(ent).build();
    }

    // ❌ Deletar NEO
    @DELETE
    @Path("/{id}")
    @Transactional
    public Response remover(@PathParam("id") Long id) {
        if (!neoService.deleteById(id))
            throw new NotFoundException("NEO não encontrado");
        return Response.noContent().build();
    }

    // 🚀 IMPORTAR DADOS DA NASA
    @POST
    @Path("/importar")
    public Response importar(
        @QueryParam("inicio") String inicio,
        @QueryParam("fim") String fim
    ) {
        LocalDate i = LocalDate.parse(inicio);
        LocalDate f = LocalDate.parse(fim);
        
        int qtd = neoService.importarFeed(i, f);

        return Response.ok(Map.of("importados", qtd)).build();
    }
}
```

### 📡 Exemplos de Uso

```bash
# Listar todos os NEOs (paginado)
GET http://localhost:8080/api/neos?pagina=0&tamanho=20

# Filtrar apenas perigosos
GET http://localhost:8080/api/neos?perigoso=true

# Obter NEO específico
GET http://localhost:8080/api/neos/123

# IMPORTAR DADOS DA NASA (1 semana)
POST http://localhost:8080/api/neos/importar
  ?inicio=2025-10-01
  &fim=2025-10-07

# Response:
{
  "importados": 42
}
```

### 🛡️ CORS Habilitado

```properties
quarkus.http.cors=true
quarkus.http.cors.origins=/.*/
quarkus.http.cors.methods=GET,POST,PUT,DELETE,OPTIONS
```

## 8️⃣ Fluxo Completo End-to-End

### 🔄 Pipeline de Dados - Visão Geral

```
┌──────────────────────────────────────────────────────────────────┐
│                    FLUXO COMPLETO DE DADOS                       │
└──────────────────────────────────────────────────────────────────┘

1️⃣ COLETA
   Frontend/Usuario
        │
        ▼ POST /api/neos/importar?inicio=2025-10-01&fim=2025-10-07
   NeoController
        │
        ▼ importarFeed()
   NeoService
        │
        ▼ buscarFeed()
   NeoWsClient ────────────► NASA NeoWs API
        │                    https://api.nasa.gov/neo/rest/v1/feed
        ▼ FeedResponse (JSON complexo)

2️⃣ TRANSFORMAÇÃO
   NeoService
        │
        ├─► mapper.writeValueAsString() → JSON completo
        │
        ├─► minioService.salvarCsvBruto() → CSV para ML
        │       │
        │       ▼ neos-2025-10-01.csv
        │   MinIO/S3
        │
        └─► normalizarPersistir() → Entidades relacionais
                │
                ▼ NeoObject entities

3️⃣ PERSISTÊNCIA
   Hibernate/Panache
        │
        ▼ merge() / upsert
   PostgreSQL
        └─► Tabela: neo_object (42 registros inseridos)

4️⃣ CONSUMO (FRONTEND)
   GET /api/neos?perigoso=true
        │
        ▼ Query: WHERE eh_potencialmente_perigoso = true
   PostgreSQL
        │
        ▼ List<NeoObject>
   NeoController
        │
        ▼ List<NeoObjectResponse> (JSON)
   Frontend (HTML/JS)

5️⃣ CONSUMO (MACHINE LEARNING)
   modelo-ia microservice
        │
        ▼ POST /ml/train/all
   MLTrainingService
        │
        ▼ listarTodosOsCsvs()
   MinIO/S3
        │
        ▼ neos-2025-10-*.csv (múltiplos arquivos)
   MLTrainingService
        │
        ▼ consolidarCsvs() → arquivo único
        ▼ treinarModelo() → Random Forest + Cost-Sensitive
        ▼ salvarModelo() → weka-model-*.model
   MinIO/S3
```

### 📊 Exemplo de Execução Real

```bash
# 1. Iniciar neo-core
./mvnw quarkus:dev -pl neo-core

# 2. Importar dados da NASA (7 dias)
curl -X POST 'http://localhost:8080/api/neos/importar?inicio=2025-10-01&fim=2025-10-07'

# Output:
# {"importados": 42}

# 3. Verificar no PostgreSQL
psql -U neo -d neo -c "SELECT COUNT(*) FROM neo_object;"
# 42

# 4. Verificar no MinIO
mc ls local/neo/
# [2025-10-07 14:30:15] 12KB neos-2025-10-01.csv
# [2025-10-07 14:30:16] 15KB neos-2025-10-02.csv
# ...

# 5. Consumir API (Frontend)
curl 'http://localhost:8080/api/neos?perigoso=true&tamanho=5'

# 6. Treinar modelo ML (modelo-ia)
curl -X POST 'http://localhost:8081/ml/train/all'
```

### ✅ Garantias de Qualidade

✅ **Idempotência**: Importar mesmos dados = upsert (sem duplicatas)  
✅ **Atomicidade**: `@Transactional` garante rollback em falhas  
✅ **Rastreabilidade**: `origemJsonS3Key` vincula dado ao CSV original  
✅ **Validação**: Try-catch para dados inválidos (null-safe)  
✅ **Performance**: Índices no banco + paginação na API  
✅ **Observabilidade**: Logs estruturados com Quarkus Logging

## 🏆 Conclusão

### 📈 Resultados Alcançados

✅ **Integração completa** com NASA NeoWs API  
✅ **Pipeline robusto** de ETL (Extract, Transform, Load)  
✅ **Persistência escalável** em PostgreSQL  
✅ **Exportação automatizada** para CSV (ML-ready)  
✅ **Armazenamento distribuído** em MinIO/S3  
✅ **API RESTful** para frontend  
✅ **Arquitetura desacoplada** (microserviços)  

### 💡 Diferenciais Técnicos

1. **Normalização Inteligente**
   - JSON aninhado → Modelo relacional
   - Tratamento de dados inválidos
   - Upsert automático (sem duplicatas)

2. **Dual Storage**
   - PostgreSQL: Dados normalizados (queries)
   - MinIO: CSV bruto (ML training)

3. **Rastreabilidade Total**
   - Campo `origemJsonS3Key`
   - Auditoria completa
   - Versionamento temporal

4. **Cloud-Native**
   - Quarkus (startup em milissegundos)
   - S3-compatible (fácil migração AWS)
   - Stateless (escalável horizontalmente)

### 🚀 Impacto no Sistema

O **neo-core** é a **fundação** do sistema NEO:

- 📥 **Alimenta** o modelo de ML com dados históricos
- 🎨 **Disponibiliza** dados para visualização no frontend
- 📊 **Centraliza** toda a lógica de negócio de coleta
- 🔄 **Automatiza** o pipeline de dados

### 📚 Tecnologias Demonstradas

- ☕ Java 17 (Records, Pattern Matching, etc.)
- 🚀 Quarkus (Framework supersônico)
- 🗄️ Hibernate ORM + Panache
- 🛢️ PostgreSQL + Flyway
- 📦 MinIO (S3-compatible)
- 📡 MicroProfile REST Client
- 🎯 JAX-RS (RESTful API)
- 🔄 Jackson (JSON/CSV)

---

## 🎬 Fim do Storytelling - neo-core

**Este módulo demonstra:**
- 📚 Integração com APIs externas (NASA)
- 🔄 ETL robusto e escalável
- 🗄️ Modelagem de dados relacional
- 📦 Armazenamento distribuído
- 🎯 API REST profissional
- 🏗️ Arquitetura de microserviços

---

### 📞 Contato
**Projeto NEO - FIAP Pós-Graduação**  
Backend de Coleta e Gerenciamento de Dados de Asteroides